In [ ]:
%matplotlib inline
import matplotlib 
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import os

import cv2

DETECTIONS = "/Users/vgenty/Desktop/rcnn/detections_7/"
VALID = "/Users/vgenty/Desktop/rcnn/valid/"
dets = os.listdir(DETECTIONS)

In [ ]:
idx_to_class = {1 : 'eminus',
                2 : 'proton',
                3 : 'pizero',
                4 : 'muminus',
                5 : 'gamma',
                6 : 'piminus',
                7 : 'kminus'}

In [ ]:
results = {'eminus' : [], 'proton' : [], 'pizero' : [], 'muminus' : [], 'gamma' : [],'piminus' : [],'kminus' : []}

for ix, det in enumerate(dets):
    d_truth  = det.split("_")[-1][:-4]
    dets_df  = pd.read_csv(DETECTIONS+det,names=['prob','xmin','ymin','xmax','ymax'])
    top      = dets_df.sort(columns='prob',ascending=False).iloc[0]
    results[d_truth].append({'name':top.name,'prob':top.prob})

In [ ]:
classifieds = { 'eminus' : {},
                'proton' : {},
                'pizero' : {},
                'muminus': {},
                'gamma' : {},
                'piminus' : {},
                'kminus' : {}}

for thresh in np.arange(0,1.01,0.01):
    for particle in classifieds :
        par = results[particle]
        for res in par:
            
            if float(res['prob'] < thresh):
                continue
                
            if thresh not in classifieds[particle]:
                classifieds[particle][thresh] = [0 for _ in xrange(7)]
                
            classifieds[particle][thresh][res['name'] - 1] += 1


p_df = pd.DataFrame(classifieds)

In [ ]:
matplotlib.rcParams['font.size'] = 16

for par in results:

    thresh_0_sum =  float(np.array(p_df[par].ix[0.0]).sum())

    fig,ax = plt.subplots(figsize=(10,6))
    for i in xrange(7):
        ax.plot(p_df.index.values,
                (p_df[par].iloc[:].str[i].values / thresh_0_sum) * 100.0,
                '-o',
                label=idx_to_class[i+1])

    ax.set_ylabel('Classified % by RCNN'.format(par),fontweight='bold')
    ax.set_xlabel('Detection prob',fontweight='bold')
    ax.set_title("Truth: {}\nSample Size: {}".format(par,thresh_0_sum))
    ax.set_ylim(0,100.0)
    ax.legend(loc='best',fontsize=12)
    plt.savefig('eff_7_class_{}.pdf'.format(par), format='pdf', dpi=1000)
    plt.show()

In [ ]:
import matplotlib.gridspec as gridspec

matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['font.family'] = 'serif'
for par in results:
    
    sums = np.array([float(np.sum(j)) for j in p_df[par].values])
    fig,_ = plt.subplots(figsize=(16,8))
    
    gs = gridspec.GridSpec(2, 1,
                           width_ratios=[1,1],
                           height_ratios=[1,2])
    
    ax = plt.subplot(gs[0])
    ax.plot(p_df.index.values,sums,'-',color='black',lw=2)
    ax.set_ylabel('N Images'.format(par),fontweight='bold')
    ax.set_title("Purity of truth: {}\n".format(par))
    ax.set_xticklabels([])
    ax = plt.subplot(gs[1])
    for i in xrange(7):
        ax.plot(p_df.index.values,
                (p_df[par].iloc[:].str[i].values / sums)*100.0,
                '-o',
                label=idx_to_class[i+1])
    
    ax.set_ylabel('Classified % by RCNN'.format(par),fontweight='bold')
    ax.set_xlabel('Detection prob treshold',fontweight='bold')
   
    ax.legend(loc='best',fontsize=12)
    plt.savefig('purity_7_class_{}.pdf'.format(par), format='pdf', dpi=1000)
    ax.set_ylim(0,100)
    plt.tight_layout()
    plt.show()